In [7]:
import glob
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [8]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
from sklearn.metrics import log_loss

In [14]:
sys.path.append('../..')
from helpers.metrics import brier_multi, CalibrationErrors, classification_error
from helpers.settings import models_folder

## Load Data

In [15]:
from keras.datasets import cifar10
from keras.utils import to_categorical

def load_cifar():
    num_classes = 10
    (x_train_val, y_train_val), (x_test, y_test) = cifar10.load_data()

    x_train_val = x_train_val.astype('float32')
    x_test = x_test.astype('float32')

    x_train_val /= 255
    x_test /= 255

    y_train_val = to_categorical(y_train_val, num_classes)
    y_test = to_categorical(y_test, num_classes)

    x_train = x_train_val[:45000]
    y_train = y_train_val[:45000]
    x_val = x_train_val[45000:]
    y_val = y_train_val[45000:]
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

In [16]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_cifar()

## Model
* The model is similar to the one that was used in Deep Ensembles and XXX paper
* TODO: try a version of the model without batch normalization and study the effect

In [17]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers.core import Lambda
from keras.layers import Conv2D, MaxPooling2D, Flatten, Activation, Dropout

from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K

In [18]:
def cnn():
    img_rows, img_cols = 32, 32
    input_shape = (img_rows, img_cols, 3)
    
    model = Sequential()
    
    model.add(Conv2D(16, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model

In [19]:
n_models = 25
epochs = 15
learning_rate = 2.5e-4
output_folder = os.path.join(models_folder, 'cifar10_cnn')
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)
#
output_baseline_folder = os.path.join(output_folder, 'baselines')
if not os.path.isdir(output_baseline_folder):
    os.makedirs(output_baseline_folder)

In [20]:
for i in range(n_models):
    seed = i+17
    # change the random state
    np.random.seed(seed)
    # creating output folder for each of the baseline models
    output_model_folder = os.path.join(output_baseline_folder, str(seed))
    if not os.path.isdir(output_model_folder):
        os.mkdir(output_model_folder)
    else:
        print('skip training...')
        continue
    # callbacks for training
    callbacks = list()
    log = CSVLogger(os.path.join(output_model_folder, 'log.csv'))
    callbacks.append(log)
    checkpoint_path = os.path.join(output_model_folder, 'weights.{epoch:02d}.hdf5')
    checkpoint = ModelCheckpoint(checkpoint_path, period=1, save_best_only=False, verbose=True)
    callbacks.append(checkpoint)
    # initialize and compile the model
    model = cnn()
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])
    # train
    model.fit(x_train, y_train, batch_size=128, epochs=epochs, verbose=1, callbacks=callbacks, validation_data=(x_val, y_val), shuffle=True)

skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...


In [ ]:
for i in range(17, 17+n_models):
    log = pd.read_csv(os.path.join(output_baseline_folder, str(i), 'log.csv'))
    plt.figure(figsize=(8,2),dpi=100)
    plt.subplots_adjust(wspace=0.5)
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    for metric_type in ['', 'val_']:
        for key in log.keys():
            if 'loss' in key and 'val' not in key:
                p = ax1.plot(log[metric_type+ key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax1.plot(np.argmin(log[metric_type+key]), np.amin(log[metric_type+key]), 
                         '*', markersize=10, alpha=0.5, color = c)
                ax1.set_title('min val NLL: {0:.3f}'.format(np.amin(log['val_' + key])))
                ax1.set_ylabel('NLL')
                ax1.set_xlabel('epochs')
            if 'acc' in key and 'val' not in key:
                p = ax2.plot(log[metric_type + key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax2.plot(np.argmax(log[metric_type + key]), np.amax(log[metric_type + key]), 
                         '*', markersize=10, alpha=0.5, color=c)
                ax2.set_title('max val acc: {0:.2f}'.format(np.amax(log['val_' + key]*100)))
                ax2.set_ylabel('accuracy')
                ax2.set_xlabel('epochs')
    ax1.grid()
    ax2.grid()
    ax1.legend()
    ax2.legend()

In [15]:
for index, model_id in enumerate(range(17, 17+n_models)):
    model_folder = os.path.join(output_baseline_folder, str(model_id))
    test_pred_path = os.path.join(model_folder, 'test_pred.npy')
    if not os.path.isfile(test_pred_path):
    # if True:
        model_folder = os.path.join(output_baseline_folder, str(model_id))
        checkpoint_path = glob.glob(model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        # checkpoint_path = glob.glob(model_folder + '/weights.' + str(best_epochs[index]).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        print('inference...')
        test_pred = model.predict(x_test)
        np.save(test_pred_path, test_pred)

In [16]:
sample_model_id = 17
y_pred_sample = np.load(os.path.join(output_baseline_folder, str(sample_model_id), 'test_pred.npy'))
shape = np.shape(y_pred_sample)
all_test_preds = np.zeros((n_models, *shape))

In [17]:
d = []
for index, model_id in enumerate(range(17, 17+n_models)):
    y_pred = np.load(os.path.join(output_baseline_folder, str(model_id), 'test_pred.npy'))
    all_test_preds[index] = y_pred
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_baselines = pd.DataFrame(d)

In [18]:
df_baselines

,model,nll,brier,ece,mce,classification error
0,17,1.049376,0.467641,11.006592,17.914299,32.97
1,18,1.120071,0.492364,12.722073,22.916308,34.26
2,19,1.081817,0.472275,10.947023,19.122961,33.28
3,20,1.035384,0.459745,10.809647,19.084227,32.63
4,21,1.129620,0.488162,13.275932,26.464154,34.49
5,22,1.046511,0.461049,12.011800,19.272041,32.47
6,23,1.049194,0.463736,11.148562,19.476030,32.28
7,24,1.030083,0.459401,10.782473,18.340129,32.60
8,25,1.054359,0.465495,12.315030,20.277822,32.66
9,26,1.080598,0.472686,11.697054,18.425075,33.13


In [19]:
print('Baseline Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_baselines[metric]), np.std(df_baselines[metric])))
    print('-'*20)

BASELINE RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
1.063 ± 0.03
--------------------
BRIER
0.469 ± 0.01
--------------------
ECE
11.718 ± 0.72
--------------------
MCE
22.828 ± 11.87
--------------------
CLASSIFICATION ERROR
33.023 ± 0.68
--------------------


# Deep Ensembles

## TODO: Increase number of models and do bootstrapping for DE to provide std...

In [20]:
n_bootstrap = 100
m_deep_ensemble = 10

In [21]:
d = []
for i in range(n_bootstrap):
    model_indices = np.random.choice(np.shape(all_test_preds)[0], size=m_deep_ensemble, replace=False)
    y_pred = np.mean(np.stack(all_test_preds[model_indices], axis=0), axis=0)
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_deep_ensembles = pd.DataFrame(d)

In [22]:
df_deep_ensembles

,model,nll,brier,ece,mce,classification error
0,41,0.703052,0.334290,8.605198,19.944169,22.82
1,41,0.707648,0.335802,8.541378,19.289700,23.04
2,41,0.706949,0.334073,8.845972,15.891629,22.94
3,41,0.709211,0.335223,9.293659,17.280933,22.54
4,41,0.706328,0.334914,8.730393,15.190134,22.87
...,...,...,...,...,...,...
95,41,0.705287,0.333100,8.951728,25.695055,22.66
96,41,0.704011,0.333073,8.804734,19.282881,22.72
97,41,0.711918,0.336928,8.960777,14.611319,22.93
98,41,0.706945,0.334302,8.519659,15.706914,23.08


In [23]:
print('Deep Ensemble Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_deep_ensembles[metric]), np.std(df_deep_ensembles[metric])))
    print('-'*20)

DEEP ENSEMBLE RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.709 ± 0.00
--------------------
BRIER
0.336 ± 0.00
--------------------
ECE
8.862 ± 0.25
--------------------
MCE
18.757 ± 10.50
--------------------
CLASSIFICATION ERROR
22.888 ± 0.19
--------------------


# Temperature Scaling

In [24]:
from scipy.optimize import minimize
def softmax_t(y_logit, t):
    return np.exp(y_logit/t)/np.sum(np.exp(y_logit/t), axis=-1)[:, np.newaxis]


def ll_t(t):
    y_val_pred = model.predict(x_val)
    y_pred_base_logit = np.log(y_val_pred)
    y_pred_temp = softmax_t(y_pred_base_logit, t)
    ll = log_loss(y_val, y_pred_temp)
    return ll

In [25]:
optimal_temps = []
optimal_temps_path = os.path.join(output_baseline_folder, 'optimal_temps.csv')
if not os.path.isfile(optimal_temps_path):
# if True:
    for index, model_id in enumerate(range(17, 17+ n_models)):
        print('finding optimal sigma for model {}...'.format(model_id))
        model_folder = os.path.join(output_baseline_folder, str(model_id))
        checkpoint_path = glob.glob( model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights...')
        model.load_weights(checkpoint_path)
        print('running optimization...')
        xopt = minimize(ll_t, 1, method='bfgs', options={'disp': 1})
        optimal_temp = xopt['x'][0]
        print('-'*100)
        print(model_id, optimal_temp)
        optimal_temps.append(optimal_temp)
    pd.DataFrame(optimal_temps, columns=['optimal temp']).to_csv(optimal_temps_path, index=False)
else:
    optimal_temps = pd.read_csv(optimal_temps_path).values
print(optimal_temps)

[[1.52187162]
 [1.60517768]
 [1.54157578]
 [1.51044265]
 [1.60736797]
 [1.56715476]
 [1.53729228]
 [1.49686581]
 [1.61902562]
 [1.58160925]
 [1.56568306]
 [1.54354629]
 [1.54358672]
 [1.55429627]
 [1.59015824]
 [1.5591671 ]
 [1.65176978]
 [1.48680123]
 [1.53217427]
 [1.54044825]
 [1.60382846]
 [1.5598986 ]
 [1.57179334]
 [1.60726693]
 [1.49423654]]


In [26]:
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_folder = os.path.join(output_baseline_folder, str(model_id))
    output_ts_folder = os.path.join(model_folder, 'temp_scaled')
    if not os.path.isdir(output_ts_folder):
        os.mkdir(output_ts_folder)
    y_pred = np.load(os.path.join(model_folder, 'test_pred.npy'))
    optimal_temp = optimal_temps[index]
    #
    y_pred_logit = np.log(y_pred)
    y_pred_ts = softmax_t(y_pred_logit, t=optimal_temp)
    #
    np.save(os.path.join(output_ts_folder, 'test_pred_ts.npy'), y_pred_ts)

In [27]:
d = list()
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_ppe_folder= os.path.join(output_baseline_folder, str(model_id),  'temp_scaled')
    y_pred = np.load(os.path.join(model_ppe_folder, 'test_pred_ts.npy'))
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_ts= pd.DataFrame(d)

In [28]:
df_ts

,model,nll,brier,ece,mce,classification error
0,17,0.958457,0.447446,0.990291,30.908940,32.97
1,18,0.992839,0.464118,1.559196,4.011182,34.26
2,19,0.980082,0.454177,1.329692,6.075970,33.28
3,20,0.947025,0.440444,1.400703,19.024498,32.63
4,21,0.998796,0.461397,1.930012,12.155861,34.49
5,22,0.934973,0.438440,1.647617,6.757016,32.47
6,23,0.945599,0.442241,1.279003,4.768501,32.28
7,24,0.942112,0.440862,1.024590,4.919921,32.60
8,25,0.937399,0.441105,0.879876,6.105325,32.66
9,26,0.967025,0.450670,1.112539,14.335243,33.13


In [29]:
print('Temperature Scaling Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_ts[metric]), np.std(df_ts[metric])))
    print('-'*20)

TEMPERATURE SCALING RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.956 ± 0.02
--------------------
BRIER
0.447 ± 0.01
--------------------
ECE
1.318 ± 0.26
--------------------
MCE
10.748 ± 7.48
--------------------
CLASSIFICATION ERROR
33.023 ± 0.68
--------------------


## PPE

In [30]:
class PPE:
    def __init__(self, model, n_ensemble):
        self.model = model
        self.n_ensemble = n_ensemble
        self.original_weights = model.get_weights()
        
    def perturb(self, sigma):
        self.model.set_weights(self.original_weights)
        weights = self.model.get_weights()
        for index2, w in enumerate(weights):
            shape = w.shape
            if len(shape) >= 2  and shape[-1] > 32:
                if len(shape) == 4:
                    noise = np.random.normal(0, sigma, (w.shape[0], w.shape[1], w.shape[2], w.shape[3]))
                else:
                    noise = np.random.normal(0, sigma, (w.shape[0], w.shape[1]))
                weights[index2] = weights[index2] + noise
        self.model.set_weights(weights)
    
    def ensemble_pred(self, x, sigma):
        y_pred = self.model.predict(x)
        y_pred_ensemble = np.zeros((self.n_ensemble, *y_pred.shape))
        for seed_index, seed in enumerate(range(17, 17 + self.n_ensemble)):
            np.random.seed(seed)
            self.perturb(sigma)
            y_pred = self.model.predict(x)
            y_pred_ensemble[seed_index] = y_pred
        self.model.set_weights(self.original_weights)
        return np.mean(y_pred_ensemble, axis=0)

In [31]:
def golden_search(model, sigma_lower=0, sigma_upper=0.1, n_ppe_ensemble=10, n_iterations=10):
    sigmas_log = list()
    nlls_log = list()
    nlls = dict()
    PHI = (np.sqrt(5) - 1) / 2
    for iteration in range(n_iterations):
        print('iteration {} {} {} '.format(iteration, sigma_lower, sigma_upper) + '-'* 10)
        sigma_1 = sigma_lower + (sigma_upper - sigma_lower) * PHI
        sigma_2 = sigma_lower + (sigma_1 - sigma_lower) * PHI
        sigmas_log.append(OrderedDict({"iteration": iteration, "sigma_l": sigma_lower,
                                       "sigma_2": sigma_2, "sigma_1": sigma_1,
                                       "sigma_h": sigma_upper}))
        for sigma in [sigma_1, sigma_2]:
            if sigma not in nlls.keys():
                ppe = PPE(model, n_ensemble=n_ppe_ensemble)
                y_val_ppe = ppe.ensemble_pred(x_val, sigma)
                nll = log_loss(y_val, y_val_ppe)
                nlls[sigma] = nll
                print(sigma, nll)
                nlls_log.append(OrderedDict({"iteration": iteration, "sigma": sigma, "nll": nll}))
            else:
                print('will skip for {0:6f}'.format(sigma))
        if nlls[sigma_1] < nlls[sigma_2]:
            sigma_lower = sigma_2
            print('updated lower side...')
        else:
            sigma_upper = sigma_1
            print('updated upper side...')
    sigma_optimal = (sigmas_log[-1]['sigma_l'] + sigmas_log[-1]['sigma_h'])/2
    return sigma_optimal

In [32]:
optimal_sigmas = []
optimal_sigmas_path = os.path.join(output_baseline_folder, 'golden_optimization_simgas.csv')
if not os.path.isfile(optimal_sigmas_path):
# if True:
    for index, model_id in enumerate(range(17, 17 + n_models)):
        print('finding optimal sigma for model {}...'.format(model_id))
        model_folder = os.path.join(output_baseline_folder, str(model_id))
        checkpoint_path = glob.glob( model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights...')
        model.load_weights(checkpoint_path)
        print('running optimization...')
        optimal_sigma = golden_search(model)
        optimal_sigmas.append(optimal_sigma)
    df = pd.DataFrame(optimal_sigmas, columns=['optimal sigma'])
    df.to_csv(optimal_sigmas_path, index=False)
else:
    df = pd.read_csv(optimal_sigmas_path)
optimal_sigmas = df['optimal sigma'].values
print(optimal_sigmas)

[0.02294902 0.02294902 0.02294902 0.01950483 0.02720626 0.02163346
 0.02294902 0.01818927 0.02213595 0.02294902 0.02082039 0.02163346
 0.02294902 0.02163346 0.02376208 0.02376208 0.02426458 0.02163346
 0.02082039 0.02213595 0.02294902 0.02376208 0.02213595 0.02294902
 0.02213595]


In [33]:
print(optimal_sigmas)

[0.02294902 0.02294902 0.02294902 0.01950483 0.02720626 0.02163346
 0.02294902 0.01818927 0.02213595 0.02294902 0.02082039 0.02163346
 0.02294902 0.02163346 0.02376208 0.02376208 0.02426458 0.02163346
 0.02082039 0.02213595 0.02294902 0.02376208 0.02213595 0.02294902
 0.02213595]


In [34]:
n_ppe_ensemble = 25
for index, model_id in enumerate(range(17, 17+n_models)):
    model_folder = os.path.join(output_baseline_folder, str(model_id))
    output_ppe_folder = os.path.join(model_folder, 'ppe')
    if not os.path.isdir(output_ppe_folder):
        os.mkdir(output_ppe_folder)
    test_pred_path = os.path.join(output_ppe_folder, 'test_pred_ppe.npy')
    if not os.path.isfile(test_pred_path):
    # if True:
        checkpoint_path = glob.glob(model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        optimal_sigma = optimal_sigmas[index]
        ppe = PPE(model, n_ensemble=n_ppe_ensemble)
        y_pred = ppe.ensemble_pred(x_test, optimal_sigma)
        np.save(test_pred_path, y_pred)
    else:
        continue

In [35]:
d = list()
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_ppe_folder= os.path.join(output_baseline_folder, str(model_id),  'ppe')
    y_pred = np.load(os.path.join(model_ppe_folder, 'test_pred_ppe.npy'))
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_ppe = pd.DataFrame(d)

In [36]:
df_ppe

,model,nll,brier,ece,mce,classification error
0,17,0.974168,0.448255,3.563987,81.138025,32.80
1,18,1.028044,0.469613,5.869813,18.479801,34.73
2,19,1.001438,0.453893,3.249139,21.162407,33.03
3,20,0.978203,0.446278,5.432041,19.648206,32.88
4,21,1.019923,0.464015,3.197253,6.467382,34.01
5,22,0.963040,0.442762,4.933659,30.632605,32.34
6,23,0.979155,0.449929,4.559371,14.577703,32.86
7,24,0.977900,0.446258,6.019534,16.974282,32.74
8,25,0.967593,0.444355,4.820005,8.089263,32.36
9,26,0.996202,0.454179,4.246012,19.413000,33.13


In [37]:
print('PPE Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_ppe[metric]), np.std(df_ppe[metric])))
    print('-'*20)

PPE RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.982 ± 0.02
--------------------
BRIER
0.450 ± 0.01
--------------------
ECE
4.599 ± 0.82
--------------------
MCE
24.007 ± 18.33
--------------------
CLASSIFICATION ERROR
32.949 ± 0.74
--------------------


# MC Dropout

In [38]:
def cnn_mcdo(num_classes=10, dropout=True, level=0.5):
    img_rows, img_cols = 32, 32
    input_shape = (img_rows, img_cols, 3)
    
    model = Sequential()
    
    model.add(Conv2D(16, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    if dropout:
        model.add(Lambda(lambda x: K.dropout(x, level=level)))
    
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    if dropout:
        model.add(Lambda(lambda x: K.dropout(x, level=level)))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    if dropout:
        model.add(Lambda(lambda x: K.dropout(x, level=level)))
    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model

In [39]:
learning_rate = 5e-4
#
output_baseline_dropout_folder = os.path.join(output_folder, 'baselines_dropout')
if not os.path.isdir(output_baseline_dropout_folder):
    os.makedirs(output_baseline_dropout_folder)

In [40]:
for i in range(n_models):
    seed = i+17
    # change the random state
    np.random.seed(seed)
    # creating output folder for each of the baseline models
    output_model_folder = os.path.join(output_baseline_dropout_folder, str(seed))
    if not os.path.isdir(output_model_folder):
        os.mkdir(output_model_folder)
        # callbacks for training
        callbacks = list()
        log = CSVLogger(os.path.join(output_model_folder, 'log.csv'))
        callbacks.append(log)
        checkpoint_path = os.path.join(output_model_folder, 'weights.{epoch:02d}.hdf5')
        checkpoint = ModelCheckpoint(checkpoint_path, period=1, save_best_only=False, verbose=True)
        callbacks.append(checkpoint)
        # initialize and compile the model
        model = cnn_mcdo()
        model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])
        # train
        model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=1, callbacks=callbacks, validation_data=(x_val, y_val), shuffle=True)

In [ ]:
for i in range(17, 17+n_models):
    log = pd.read_csv(os.path.join(output_baseline_dropout_folder, str(i), 'log.csv'))
    plt.figure(figsize=(8,2),dpi=100)
    plt.subplots_adjust(wspace=0.5)
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    for metric_type in ['', 'val_']:
        for key in log.keys():
            if 'loss' in key and 'val' not in key:
                p = ax1.plot(log[metric_type+ key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax1.plot(np.argmin(log[metric_type+key]), np.amin(log[metric_type+key]), 
                         '*', markersize=10, alpha=0.5, color = c)
                ax1.set_title('min val NLL: {0:.3f}'.format(np.amin(log['val_' + key])))
                ax1.set_ylabel('NLL')
                ax1.set_xlabel('epochs')
            if 'acc' in key and 'val' not in key:
                p = ax2.plot(log[metric_type + key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax2.plot(np.argmax(log[metric_type + key]), np.amax(log[metric_type + key]), 
                         '*', markersize=10, alpha=0.5, color=c)
                ax2.set_title('max val acc: {0:.2f}'.format(np.amax(log['val_' + key]*100)))
                ax2.set_ylabel('accuracy')
                ax2.set_xlabel('epochs')
    ax1.grid()
    ax2.grid()
    ax1.legend()
    ax2.legend()

In [ ]:
for index, model_id in enumerate(range(17, 17+n_models)):
    model_folder = os.path.join(output_baseline_dropout_folder, str(model_id))
    test_pred_path = os.path.join(model_folder, 'test_pred.npy')
    if not os.path.isfile(test_pred_path):
        checkpoint_path = glob.glob(model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn_mcdo(dropout=False)
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        print('inference...')
        test_pred = model.predict(x_test)
        np.save(test_pred_path, test_pred)

In [43]:
sample_model_id = 17
y_pred_sample = np.load(os.path.join(output_baseline_folder, str(sample_model_id), 'test_pred.npy'))
shape = np.shape(y_pred_sample)
all_test_preds = np.zeros((n_models, *shape))

In [44]:
d = []
for index, model_id in enumerate(range(17, 17 + n_models)):
    y_pred = np.load(os.path.join(output_baseline_dropout_folder, str(model_id), 'test_pred.npy'))
    all_test_preds[index] = y_pred
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_baselines_do = pd.DataFrame(d)

In [45]:
print('Baseline Dropout Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_baselines_do[metric]), np.std(df_baselines_do[metric])))
    print('-'*20)

BASELINE DROPOUT RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
1.054 ± 0.09
--------------------
BRIER
0.481 ± 0.03
--------------------
ECE
4.747 ± 1.67
--------------------
MCE
12.815 ± 7.18
--------------------
CLASSIFICATION ERROR
35.507 ± 3.09
--------------------


In [46]:
n_mcdo_ensemble = 25
for index, model_id in enumerate(range(17, 17+n_models)):
    output_mcdo_folder = os.path.join(model_folder, 'mcdo')
    test_pred_path = os.path.join(output_mcdo_folder, 'test_pred_mcdo.npy')
    # if True:
    if not os.path.isfile(test_pred_path):
        model_folder = os.path.join(output_baseline_dropout_folder, str(model_id))
        if not os.path.isdir(output_mcdo_folder):
            os.mkdir(output_mcdo_folder)
        checkpoint_path = glob.glob(model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn_mcdo()
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        y_test_pred_ensemble = np.zeros((n_mcdo_ensemble, *y_test.shape))
        for index2, seed in enumerate(range(17, 17 + n_mcdo_ensemble)):
            np.random.seed(seed)
            y_test_pred_dropout = model.predict(x_test, verbose=1)
            y_test_pred_ensemble[index2] = y_test_pred_dropout
        y_pred = np.mean(y_test_pred_ensemble, axis=0)
        np.save(test_pred_path, y_pred)

In [47]:
d = list()
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_mcdo_folder= os.path.join(output_baseline_dropout_folder, str(model_id),  'mcdo')
    y_pred = np.load(os.path.join(model_mcdo_folder, 'test_pred_mcdo.npy'))
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_mcdo= pd.DataFrame(d)

In [48]:
df_mcdo

,model,nll,brier,ece,mce,classification error
0,17,0.806739,0.384366,6.292623,11.408242,27.80
1,18,0.806345,0.384910,7.010246,14.670719,27.35
2,19,0.784237,0.376619,6.639755,10.575379,26.77
3,20,0.788280,0.376721,7.558372,13.926969,26.67
4,21,0.779271,0.372362,6.814007,13.672411,26.59
5,22,0.836244,0.396332,5.918552,12.963861,28.88
6,23,0.798381,0.382612,7.648522,12.253529,27.07
7,24,0.793404,0.378043,6.417017,14.695643,26.84
8,25,0.821689,0.394077,6.335888,10.695846,28.43
9,26,0.784066,0.375467,7.400974,11.416102,26.61


In [49]:
print('MCDO Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_mcdo[metric]), np.std(df_mcdo[metric])))
    print('-'*20)

MCDO RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.798 ± 0.01
--------------------
BRIER
0.381 ± 0.01
--------------------
ECE
7.109 ± 0.62
--------------------
MCE
15.921 ± 14.29
--------------------
CLASSIFICATION ERROR
27.207 ± 0.66
--------------------


In [50]:
dfs = [df_baselines,
df_ppe,
df_ts,
df_mcdo,
df_deep_ensembles
      ]

In [51]:
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('% {}'.format(metric))
    for index, df in enumerate(dfs):
        if index != len(dfs)-1:
            print('{0:.3f} $\pm$ {1:.2f} &'.format(np.mean(df[metric]), np.std(df[metric])))
        else:
            print('{0:.3f} $\pm$ {1:.2f} \\\\'.format(np.mean(df[metric]), np.std(df[metric])))
    print('-'*100)

% nll
1.063 $\pm$ 0.03 &
0.982 $\pm$ 0.02 &
0.956 $\pm$ 0.02 &
0.798 $\pm$ 0.01 &
0.709 $\pm$ 0.00 \\
----------------------------------------------------------------------------------------------------
% brier
0.469 $\pm$ 0.01 &
0.450 $\pm$ 0.01 &
0.447 $\pm$ 0.01 &
0.381 $\pm$ 0.01 &
0.336 $\pm$ 0.00 \\
----------------------------------------------------------------------------------------------------
% ece
11.718 $\pm$ 0.72 &
4.599 $\pm$ 0.82 &
1.318 $\pm$ 0.26 &
7.109 $\pm$ 0.62 &
8.862 $\pm$ 0.25 \\
----------------------------------------------------------------------------------------------------
% mce
22.828 $\pm$ 11.87 &
24.007 $\pm$ 18.33 &
10.748 $\pm$ 7.48 &
15.921 $\pm$ 14.29 &
18.757 $\pm$ 10.50 \\
----------------------------------------------------------------------------------------------------
% classification error
33.023 $\pm$ 0.68 &
32.949 $\pm$ 0.74 &
33.023 $\pm$ 0.68 &
27.207 $\pm$ 0.66 &
22.888 $\pm$ 0.19 \\
----------------------------------------------------